<a href="https://colab.research.google.com/github/martinpius/RNN-ALIENS/blob/main/RNN_Vanila_from_scartch_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on Google CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)}: {e}\n>>>> please correct {type(e)} and reload your device")
def time_fmt(t: float = 129.891)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h} min: {m:>02} sec: {s:>05.2f}"
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f">>>> time formating\t.................\n>>>> time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>> You are on Google CoLaB with torch version: 1.8.1+cu101
>>>> time formating	.................
>>>> time elapsed	hrs: 0 min: 02 sec: 09.00


In [ ]:
#We are going to implementing the simple vanilla RNN model. RNN normally is best for sequential data
#such as survival data, time series data, texts, sound, videos generation datasets. For demonstration
#we are going to use the famous simple deep learning data set (MNIST)by Yann-Lecun

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
import time, datetime, sys


In [ ]:
#Hyperparameters to be used in this network
input_dim = 28
seq_len = 28
EPOCHS = 10
batch_size = 128
hidden_dim = 512
learning_rate = 1e-3
num_layers = 2
num_classes = 10


In [ ]:
class V_RNN(nn.Module):
  def __init__(self,input_dim, hidden_dim, num_layers,num_classes):
    super(V_RNN, self).__init__()
    self.num_layers = num_layers
    self.hidden_dim = hidden_dim
    self.rnn = nn.RNN(input_size = input_dim,
                      hidden_size = hidden_dim,
                      num_layers = num_layers,
                      batch_first = True, 
                      dropout = 0.5)
    self.fc1 = nn.Linear(in_features = hidden_dim * seq_len, out_features = 128)
    self.fc2 = nn.Linear(in_features = 128, out_features = num_classes)
  
  def forward(self, input_tensor):
    h0 = torch.zeros(size = (self.num_layers, input_tensor.size(0), self.hidden_dim)).to(device = device)
    out, _ = self.rnn(input_tensor, h0) #The output including hidden units and cell status
    out = out.reshape(out.shape[0], -1) #Flattening the output into (batch, total features)
    x = self.fc1(out)
    x = self.fc2(x)
    return x



In [ ]:
#Instantiating and Testing the network if it gives the desired output
rand_input = torch.randn(size = (32, 28,28)).to(device = device)
model = V_RNN(input_dim = input_dim, hidden_dim = hidden_dim, num_layers = num_layers, num_classes = num_classes).to(device = device)
print(f">>>> the output shape is : {model(rand_input).shape}")

>>>> the output shape is : torch.Size([32, 10])


In [ ]:
#Fetch and preprocessing the data from torchvision:
train_dfm = datasets.MNIST(root = "mnist_train_rnn/", train = True, download = True, transform = transforms.ToTensor())
valid_dfm = datasets.MNIST(root = "mnist_test_rnn/", train = False, download = True, transform = transforms.ToTensor())
train_loader = DataLoader(dataset = train_dfm, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(dataset = valid_dfm, shuffle = True,batch_size = batch_size)
x_train_batch, y_train_batch = next(iter(train_loader))
print(f">>>> x_train_shape: {x_train_batch.shape}\ty_train_shape: {y_train_batch.shape}")

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train_rnn/MNIST/raw/train-images-idx3-ubyte.gz to mnist_train_rnn/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train_rnn/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_train_rnn/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train_rnn/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_train_rnn/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train_rnn/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_train_rnn/MNIST/raw

Processing...
Done!


...

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_test_rnn/MNIST/raw/train-images-idx3-ubyte.gz to mnist_test_rnn/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_test_rnn/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_test_rnn/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_test_rnn/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_test_rnn/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_test_rnn/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_test_rnn/MNIST/raw

Processing...
Done!
>>>> x_train_shape: torch.Size([128, 1, 28, 28])	y_train_shape: torch.Size([128])


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)

In [ ]:
#The training loop: from scratch
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>> training starts for epoch {epoch + 1}\tloading................\n>>>> please wait while the model is training................")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    data = data.squeeze(1).to(device = device)
    target = target.to(device = device)
    #forward pass
    preds = model(data)
    train_loss = loss_fn(preds, target)
    optimizer.zero_grad() #initialize the slopes to zeros
    train_loss.backward() #backward pass
    optimizer.step() #gradient descent with adam optimizer

def __monitor__(loader, model):
  if loader.dataset.train:
    print(f"\n>>>> checking accuracy over the training data\tplease wait................")
  else:
    print(f"\n>>>> checking accuracy over the validation data\tplease wait...............")
  num_examples = 0
  num_correct = 0
  model.eval() # no need to train the model once again
  with torch.no_grad():
    for (x, y) in test_loader:
      x = x.squeeze(1).to(device = device)
      y = y.to(device = device)
      preds = model(x)
      _, predictions = preds.max(1) #fetch maximum probability amoung the 10 members of the class
      num_correct+=(predictions == y).sum() # get the total number of correct predictions
      num_examples+= predictions.size(0) # get the number of total examples
      acc = float((num_correct/num_examples)*100)
      print(f"\n>>>> at epoch {epoch+1}, batch number {idx + 1}, accuracy is {acc:.4f}")
  model.train()
toc = time.time()
__monitor__(train_loader, model)
__monitor__(test_loader, model)
print(f"\n>>>> total time elapsed: {time_fmt(toc - tic)}")

    


  0%|          | 0/469 [00:00<?, ?it/s]


>>>> training starts for epoch 1	loading................
>>>> please wait while the model is training................



100%|██████████| 469/469 [03:26<00:00,  2.28it/s]

  0%|          | 0/469 [00:00<?, ?it/s]


>>>> training starts for epoch 2	loading................
>>>> please wait while the model is training................



100%|██████████| 469/469 [03:25<00:00,  2.28it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

...


100%|██████████| 469/469 [03:45<00:00,  2.08it/s]

  0%|          | 0/469 [00:00<?, ?it/s]


>>>> training starts for epoch 10	loading................
>>>> please wait while the model is training................



100%|██████████| 469/469 [03:43<00:00,  2.10it/s]


>>>> checking accuracy over the training data	please wait................


ValueError: ignored